In [ ]:
import plotly.express as px
import geopandas as gpd
import shapely.geometry
import numpy as np
import wget


# download a zipped shapefile
wget.download("https://plotly.github.io/datasets/ne_50m_rivers_lake_centerlines.zip")

# open a zipped shapefile with the zip:// pseudo-protocol
geo_df = gpd.read_file("zip://ne_50m_rivers_lake_centerlines.zip")

lats = []
lons = []
names = []

for feature, name in zip(geo_df.geometry, geo_df.name):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)

fig = px.line_geo(lat=lats, lon=lons, hover_name=names)
fig.show()

In [8]:
# mandatory if Python ssl.SSLError occurs
#import ssl
#import certifi
#ctx = ssl.create_default_context(cafile=certifi.where())
#geopy.geocoders.options.default_ssl_context = ctx
#import geopy

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='telecom_paris', timeout=7)
location = geolocator.geocode('19 Place Marguerite Perey, 91120 Palaiseau, France')
location.raw

/Users/dorianfitton/opt/anaconda3/envs/bgd/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


{'place_id': 346057888,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 9903423,
 'boundingbox': ['48.7123423', '48.7134647', '2.1988453', '2.2015918'],
 'lat': '48.7129116',
 'lon': '2.1998893037725673',
 'display_name': 'Télécom Paris, 19, Place Marguerite Perey, La Vauve - École Polytechnique, Campus Urbain de Paris-Saclay, Palaiseau, Essonne, Île-de-France, France métropolitaine, 91120, France',
 'class': 'building',
 'type': 'yes',
 'importance': 1.1725057412006172}

In [9]:
# get latitude and longitude
lat, lon = location.raw['lat'], location.raw['lon']
lat, lon

('48.7129116', '2.1998893037725673')

In [10]:
# demo


# map arround 48.71, 2.20
m = Map(center=(lat, lon), zoom=18)

# satellite map for yesterday
# NASA's Global Imagery Browse Services
yesterday = datetime.now() - timedelta(days=14)
nasa = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, yesterday.strftime('%Y-%m-%d'))
m = Map(layers=(nasa, ), center=(lat, lon), zoom=8)


df = pd.read_csv('/Users/dorianfitton/Documents/Cours_Télécom/Kit BIG DATA/kit_bgd/cities500.zip',
                 sep='\t',header=None,
                 keep_default_na=False,  # NA = North America
                 na_values=['', -9999],
                 names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude',
                        'longitude', 'feature class', 'feature code', 'country code',
                        'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code',
                        'population', 'elevation', 'dem', 'timezone', 'modification date'],
                        dtype={'admin1 code': str,
                               'admin2 code': str,
                               'admin3 code': str,
                               'admin4 code': str})
df.shape




m

In [11]:
df

,Latitude,Longitude
0,45°55.11'N,45°55.11'N
1,45°55.11'S,45°55.11'S


In [12]:
df.replace(r'([0-9]{2})(°)?(\d*)(\.?)(\d+)(\')([A-Z]?)', r'\3\4\5', regex=True)

,Latitude,Longitude
0,55.11,55.11
1,55.11,55.11


In [36]:
df.replace(r'([0-9]{2})(°)?(\d*)\.?(\d+)(\'[A-Z]?)', r'\4', regex=True)

,0
0,11
1,11


In [41]:
def dms2dec(deg, mn, cap):
    if cap in ['N', 'E']:
        x = deg + mn / 60
    if cap in ['S', 'W']:
        x = (deg + mn / 60)*-1

    return x

In [42]:
dms2dec(45, 55.11, 'N')

45.9185

In [70]:
def convert_lat_lon(df):
    deg = df['Latitude'].replace(r'(\d+)(°)?(\d*)\.?(\d+)(\'[A-Z]?)', r'\1', regex=True).astype(int)

    min = df['Latitude'].replace(r'(\d+)(°)?(\d*)(\.?)(\d+)(\')([A-Z]?)', r'\3\4\5', regex=True).astype(float)


    cap = df['Latitude'].replace(r'(\d+)(°)?(\d*)\.?(\d+)(\')([A-Z]?)', r'\6', regex=True)

    df['Latitude']=dms2dec(deg,min,cap)

    deg = df['Longitude'].replace(r'(\d+)(°)?(\d*)\.?(\d+)(\'[A-Z]?)', r'\1', regex=True).astype(int)

    min = df['Longitude'].replace(r'(\d+)(°)?(\d*)(\.?)(\d+)(\')([A-Z]?)', r'\3\4\5', regex=True).astype(float)


    cap = df['Longitude'].replace(r'(\d+)(°)?(\d*)\.?(\d+)(\')([A-Z]?)', r'\6', regex=True)

    df['Longitude']=dms2dec(deg,min,cap.apply(lambda x: x))

    return df

def dms2dec(deg, mn, cap):
    map = {'N':1, 'E':1,'S':-1, 'W':-1}
    cap_m = cap.map(map)

    x = (deg + mn / 60)*cap_m

    return x

In [69]:
cap_m

0    1
1   -1
Name: Longitude, dtype: int64

In [ ]:
    if cap_ in ['N', 'E']:
        x = deg + mn / 60
    if cap_ in ['S', 'W']:
        x = (deg + mn / 60)*-1

In [71]:
import pandas as pd
lat = ["45°55.11'N","45°55.11'N"]
lon = ["6°55.11'E","7°55.11'W"]

df=pd.DataFrame([lat,lon], columns=['Latitude', 'Longitude'])

In [66]:
df

,Latitude,Longitude
0,45.9185,-45.9185
1,6.9185,-7.9185


In [72]:
convert_lat_lon(df)

,Latitude,Longitude
0,45.9185,45.9185
1,6.9185,-7.9185


In [ ]:
deg = df['Latitude'].replace(r'([0-9]{2})(°)?(\d*)\.?(\d+)(\'[A-Z]?)', r'\1', regex=True).astype(int)

min = df['Latitude'].replace(r'([0-9]{2})(°)?(\d*)(\.?)(\d+)(\')([A-Z]?)', r'\3\4\5', regex=True).astype(float)


cap = df['Latitude'].replace(r'([0-9]{2})(°)?(\d*)\.?(\d+)(\')([A-Z]?)', r'\6', regex=True)

df['Latitude']=dms2dec(deg,min,cap)

deg = df['Longitude'].replace(r'([0-9]{2})(°)?(\d*)\.?(\d+)(\'[A-Z]?)', r'\1', regex=True).astype(int)

min = df['Longitude'].replace(r'([0-9]{2})(°)?(\d*)(\.?)(\d+)(\')([A-Z]?)', r'\3\4\5', regex=True).astype(float)


cap = df['Longitude'].replace(r'([0-9]{2})(°)?(\d*)\.?(\d+)(\')([A-Z]?)', r'\6', regex=True)

df['Longitude']=dms2dec(deg,min,cap)



In [39]:
deg = df['Latitude'].replace(r'([0-9]{2})(°)?(\d*)\.?(\d+)(\'[A-Z]?)', r'\1', regex=True).astype(int)
deg

0    45
1    45
Name: Latitude, dtype: int64

In [40]:
min = df['Latitude'].replace(r'([0-9]{2})(°)?(\d*)(\.?)(\d+)(\')([A-Z]?)', r'\3\4\5', regex=True).astype(float)
min

0    55.11
1    55.11
Name: Latitude, dtype: float64

In [41]:
cap = df['Longitude'].replace(r'([0-9]{2})(°)?(\d*)\.?(\d+)(\')([A-Z]?)', r'\6', regex=True)
cap

0    N
1    S
Name: Longitude, dtype: object

In [49]:

for cap_ in cap:
    if cap_ in ['N', 'E']:
        x = deg + min / 60
    if cap_ in ['S', 'W']:
        x = (deg + min / 60)*-1


In [50]:
x

0   -45.9185
1   -45.9185
Name: Latitude, dtype: float64

In [51]:
df

,Latitude,Longitude
0,45°55.11'N,45°55.11'N
1,6°55.11'E,7°55.11'W


In [53]:
df = pd.read_csv('/Users/dorianfitton/Documents/Cours_Télécom/Kit BIG DATA/kit_bgd/projet_final/dataset_complet')

In [56]:
df[['Longitude', 'Latitude']]

,Longitude,Latitude
0,-2.432000,46.713500
1,-3.815000,47.140667
2,-4.706000,47.182167
3,-5.344833,47.213667
4,-5.585167,47.151833
...,...,...
14204,-9.289500,45.457167
14205,-8.861333,45.930167
14206,-8.685833,46.346167
14207,-8.574833,46.910167
